In [1]:
import numpy as np
import math

In [2]:
"""
A funcao 'plota' produz um gráfico da estrutura definida pela matriz de nos N 
e pela incidencia Inc.

Sugestao de uso:

from funcoesTermosol import plota
plota(N,Inc)
-------------------------------------------------------------------------------
A funcao 'importa' retorna o numero de nos [nn], a matriz dos nos [N], o numero
de membros [nm], a matriz de incidencia [Inc], o numero de cargas [nc], o vetor
carregamento [F], o numero de restricoes [nr] e o vetor de restricoes [R] 
contidos no arquivo de entrada.

Sugestao de uso:
    
from funcoesTermosol import importa
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xlsx')
-------------------------------------------------------------------------------
A funcao 'geraSaida' cria um arquivo nome.txt contendo as reacoes de apoio Ft, 
deslocamentos Ut, forcas Fi e tensoes Ti internas. As entradas devem ser 
vetores coluna.

Sugestao de uso:
    
from funcoesTermosol import geraSaida
geraSaida(nome,Ft,Ut,Epsi,Fi,Ti)
-------------------------------------------------------------------------------

"""
def plota(N,Inc):
    # Numero de membros
    nm = len(Inc[:,0])
    
    import matplotlib as mpl
    import matplotlib.pyplot as plt

#    plt.show()
    fig = plt.figure()
    # Passa por todos os membros
    for i in range(nm):
        
        # encontra no inicial [n1] e final [n2] 
        n1 = int(Inc[i,0])
        n2 = int(Inc[i,1])        

        plt.plot([N[0,n1-1],N[0,n2-1]],[N[1,n1-1],N[1,n2-1]],color='r',linewidth=3)


    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.grid(True)
    plt.axis('equal')
    plt.show()
    
def importa(entradaNome):
    
    import numpy as np
    import xlrd
    
    arquivo = xlrd.open_workbook(entradaNome)
    
    ################################################## Ler os nos
    nos = arquivo.sheet_by_name('Nos')
    
    # Numero de nos
    nn = int(nos.cell(1,3).value)
                 
    # Matriz dos nós
    N = np.zeros((2,nn))
    
    for c in range(nn):
        N[0,c] = nos.cell(c+1,0).value
        N[1,c] = nos.cell(c+1,1).value
    
    ################################################## Ler a incidencia
    incid = arquivo.sheet_by_name('Incidencia')
    
    # Numero de membros
    nm = int(incid.cell(1,5).value)
                 
    # Matriz de incidencia
    Inc = np.zeros((nm,4))
    
    for c in range(nm):
        Inc[c,0] = int(incid.cell(c+1,0).value)
        Inc[c,1] = int(incid.cell(c+1,1).value)
        Inc[c,2] = incid.cell(c+1,2).value
        Inc[c,3] = incid.cell(c+1,3).value
    
    ################################################## Ler as cargas
    carg = arquivo.sheet_by_name('Carregamento')
    
    # Numero de cargas
    nc = int(carg.cell(1,4).value)
                 
    # Vetor carregamento
    F = np.zeros((nn*2,1))
    
    for c in range(nc):
        no = carg.cell(c+1,0).value
        xouy = carg.cell(c+1,1).value
        GDL = int(no*2-(2-xouy)) 
        F[GDL-1,0] = carg.cell(c+1,2).value
         
    ################################################## Ler restricoes
    restr = arquivo.sheet_by_name('Restricao')
    
    # Numero de restricoes
    nr = int(restr.cell(1,3).value)
                 
    # Vetor com os graus de liberdade restritos
    R = np.zeros((nr,1))
    
    for c in range(nr):
        no = restr.cell(c+1,0).value
        xouy = restr.cell(c+1,1).value
        GDL = no*2-(2-xouy) 
        R[c,0] = GDL-1


    return nn,N,nm,Inc,nc,F,nr,R

def geraSaida(nome,Ft,Ut,Epsi,Fi,Ti):
    nome = nome + '.txt'
    f = open("saida.txt","w+")
    f.write('Reacoes de apoio [N]\n')
    f.write(str(Ft))
    f.write('\n\nDeslocamentos [m]\n')
    f.write(str(Ut))
    f.write('\n\nDeformacoes []\n')
    f.write(str(Epsi))
    f.write('\n\nForcas internas [N]\n')
    f.write(str(Fi))
    f.write('\n\nTensoes internas [Pa]\n')
    f.write(str(Ti))
    f.close()
    


[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xlsx')
N

array([[ 0. ,  0. ,  0.3],
       [ 0. ,  0.4,  0.4]])

In [3]:
x = N[0]
y = N[1]


In [4]:
#descobrindo o L
L = []

for e in range(nn-1):
    L.append(((x[e]-x[e+1])**2+(y[e]-y[e+1])**2)**(1/2))
L.append(((x[0] -x[nn-1])**2+(y[0]-y[nn-1])**2)**(1/2))

L


[0.40000000000000002, 0.29999999999999999, 0.5]

In [5]:
#descobrindo os senos e cossenos
cos =[]
sen=[]

for e in range(nn-1):
    sen.append((y[e+1]-y[e])/L[e])
    cos.append((x[e+1]-x[e])/L[e])

cos.append((x[nn-1]-x[0])/L[nn-1])
sen.append((y[nn-1]-y[0])/L[nn-1])



In [6]:
E = []
A = []

for e in range(nn):
    E.append(Inc[e][2])
    A.append(Inc[e][3])

In [9]:
TK =[]

for e in range(nn):
    c = cos[e]
    s = sen [e]
    T = np.array([[c**2, c*s, -c**2, -c*s],
                [c*s, s**2, -c*s, -s**2],
                 [-c**2, -c*s, c**2, c*s],
                 [-c*s, -s**2, c*s, s**2]])
    TK.append((E[e]*A[e]/L[e]) * (T))
    
    
TK

[array([[  0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
          -0.00000000e+00],
        [  0.00000000e+00,   1.05000000e+08,  -0.00000000e+00,
          -1.05000000e+08],
        [ -0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [ -0.00000000e+00,  -1.05000000e+08,   0.00000000e+00,
           1.05000000e+08]]),
 array([[  1.40000000e+08,   0.00000000e+00,  -1.40000000e+08,
          -0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
          -0.00000000e+00],
        [ -1.40000000e+08,  -0.00000000e+00,   1.40000000e+08,
           0.00000000e+00],
        [ -0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
           0.00000000e+00]]),
 array([[ 30240000.        ,  40320000.        , -30240000.        ,
         -40320000.        ],
        [ 40320000.        ,  53760000.00000001, -40320000.        ,
         -53760000.00000001],
        [-30240000.        , -40320000.        ,  30240000.        ,
 

In [114]:
gdlmax = 6

gdl = np.array([[0,1,2,3],
               [2,3,4,5],
               [4,5,0,1]])

Kglobal = np.zeros((gdlmax,gdlmax))

listauxiliar = []
for e in range(len(gdl)):
    listauxiliar = gdl[e]
    for i in range(0,4):
        for j in range(0,4):
            Kglobal[listauxiliar[i]][listauxiliar[j]] += TK[e][i][j]
    
Kglobal

array([[  3.02400000e+07,   4.03200000e+07,   0.00000000e+00,
          0.00000000e+00,  -3.02400000e+07,  -4.03200000e+07],
       [  4.03200000e+07,   1.58760000e+08,   0.00000000e+00,
         -1.05000000e+08,  -4.03200000e+07,  -5.37600000e+07],
       [  0.00000000e+00,   0.00000000e+00,   1.40000000e+08,
          0.00000000e+00,  -1.40000000e+08,   0.00000000e+00],
       [  0.00000000e+00,  -1.05000000e+08,   0.00000000e+00,
          1.05000000e+08,   0.00000000e+00,   0.00000000e+00],
       [ -3.02400000e+07,  -4.03200000e+07,  -1.40000000e+08,
          0.00000000e+00,   1.70240000e+08,   4.03200000e+07],
       [ -4.03200000e+07,  -5.37600000e+07,   0.00000000e+00,
          0.00000000e+00,   4.03200000e+07,   5.37600000e+07]])

In [124]:
Pg = np.array ([[0],
            [150],
            [-100]])


Pgindice = [1,4,5]

In [128]:
Kcontorno = np.zeros((len(Pgindice),len(Pgindice)))

for e in range(len(Pgindice)):
    for x in range(len(Pgindice)):
        Kcontorno[e][x] += Kglobal[Pgindice[e]][Pgindice[x]]

Kcontorno

array([[  1.58760000e+08,  -4.03200000e+07,  -5.37600000e+07],
       [ -4.03200000e+07,   1.70240000e+08,   4.03200000e+07],
       [ -5.37600000e+07,   4.03200000e+07,   5.37600000e+07]])

In [130]:
np.linalg.inv(Kcontorno) * Kcontorno

array([[  1.51200000e+00,   1.41355009e-18,  -5.12000000e-01],
       [ -1.06870691e-18,   1.21600000e+00,  -2.16000000e-01],
       [ -5.12000000e-01,  -2.16000000e-01,   1.72800000e+00]])

In [135]:
uzinho = np.dot(np.linalg.inv(Kcontorno), Pg)
uzinho

array([[ -9.52380952e-07],
       [  1.60714286e-06],
       [ -4.01785714e-06]])

In [166]:
Uzao = np.zeros((gdlmax,1))

for e in range(len(Pgindice)):
    Uzao[Pgindice[e]][0] += uzinho[e][0]

Uzao

array([[  0.00000000e+00],
       [ -9.52380952e-07],
       [  0.00000000e+00],
       [  0.00000000e+00],
       [  1.60714286e-06],
       [ -4.01785714e-06]])